In [1]:
import gradio as gr
import pandas as pd
import os
import tempfile
import time
import playsound
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from dotenv import load_dotenv
from transformers import pipeline
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import APIChain
from gtts import gTTS

from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

import sentiment_analysis

In [2]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
GEMINI_MODEL = "gemini-1.5-flash"

# Store the API key in a variable.
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [3]:
# Text description of API spec.
spec_str = """

Instructions for responding:

Use the Open Library Search API to get up to 4 books related to the person the book is requested for.  
When responding pull any 4 random books from the list do not include any books the users asks to exclude. 
Provide the response to the user with the title and author of the books in natural language. 

When a user asks for information about a book by the title:
use Open Library Search API to get the description of the book between 100 and 200 words in length, if there
is no description available tell the user a description isn't available and list 4 similar books.

When a user asks for information about the author:
use Open Library Search API to get information about the author.
Respond to the user by telling them about the author and 
use Open Library Search API to provide 2 more books written by that author.

Open Library provides an experimental API to search.

WARNING: This API is under active development and may change in future.

Overview & Features
The Open Library Search API is one of the most convenient and complete ways to retrieve book data on Open Library. The API:

Is able to return data for multiple books in a single request/response
Returns both Work level information about the book, as well as Edition level information (such as)
Author IDs are returned which you can use to fetch the author's image, if available
Options are available to return Book Availability along with the response.
Powerful sorting options are available, such as star ratings, publication date, and number of editions.
Endpoint
The endpoint for this API is:
https://openlibrary.org/search.json

Examples
The URL format for API is simple. Take the search URL and add .json to the end. Eg:

https://openlibrary.org/search.json?q=the+lord+of+the+rings
https://openlibrary.org/search.json?title=the+lord+of+the+rings
https://openlibrary.org/search.json?author=tolkien&sort=new
https://openlibrary.org/search.json?q=the+lord+of+the+rings&page=2
https://openlibrary.org/search/authors.json?q=twain
Using Thing IDs to get Images
You can use the olid (Open Library ID) for authors and books to fetch covers by olid, e.g.:
https://covers.openlibrary.org/a/olid/OL23919A-M.jpg

URL Parameters
Parameter	Description
q	The solr query. See Search HowTo for sample queries
fields	The fields to get back from solr. Use the special value * to get all fields (although be prepared for a very large response!).
To fetch availability data from archive.org, add the special value, availability. Example: /search.json?q=harry%20potter&fields=*,availability&limit=1. This will fetch the availability data of the first item in the `ia` field.
sort	You can sort the results by various facets such as new, old, random, or key (which sorts as a string, not as the number stored in the string). For a complete list of sorts facets look here (this link goes to a specific commit, be sure to look at the latest one for changes). The default is to sort by relevance.
lang	The users language as a two letter (ISO 639-1) language code. This influences but doesn't exclude search results. For example setting this to fr will prefer/display the French edition of a given work, but will still match works that don't have French editions. Adding language:fre on the other hand to the search query will exclude results that don't have a French edition.
offset / limit	Use for pagination.
page / limit	Use for pagination, with limit corresponding to the page size. Note page starts at 1.
Include no other text, only the API call URL. Don't use newlines."""

In [4]:
# Initialize the model.
llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.9)
moody_llm = ChatGoogleGenerativeAI(google_api_key=GEMINI_API_KEY, model=GEMINI_MODEL, temperature=0.9)

# Initialize an object for conversational memory.
buffer = ConversationBufferMemory()

# Create the chain for conversation, using a ConversationBufferMemory object.
conversation = ConversationChain(llm=moody_llm, verbose=True, memory=buffer)

/var/folders/f6/_37vjycn6436mlkxtc3jkqdw0000gn/T/ipykernel_5258/139866355.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer = ConversationBufferMemory()
/var/folders/f6/_37vjycn6436mlkxtc3jkqdw0000gn/T/ipykernel_5258/139866355.py:9: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(llm=moody_llm, verbose=True, memory=buffer)


In [5]:
# Create a function that takes in a message and executes the openlibrary.org API
def run_api(msg, history):
     # Define a query as a dictionary using the user's input.
    query = {"question": msg}


# Create the API chain from the spec, using the LLM.
    chain = APIChain.from_llm_and_api_docs(
    llm,
    spec_str,
    #verbose = True,
    limit_to_domains=["https://openlibrary.org/"],
    )
    try:
        # Run the chain using the query, and print the response.
        response = chain.invoke(query)
        #print(response["output"])
        return response["output"]
    except:
        return "Sorry, that query generated too large of a result. Please try a different question."

In [6]:
# Function is run if the user enters a very negative message.
def run_moody(msg, history):
    result = conversation.predict(input=msg)
    #print(result)
    return result

In [7]:
# Flag is set if the negative sentiment is activated, keeps user in that LLM
neg_flag = False

In [8]:
def text_to_speech(message):
    try:
        temp_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        # Convert the text response to speech.
        tts = gTTS(message, lang='en')        
        tts.save(temp_file.name)
        return temp_file.name
    except Exception as e:
        error_message = f"Sorry, an error occurred: {e}. Please try again."
        tts = gTTS(error_message, lang='en')
        tts.save(temp_file.name)
        return None

In [9]:
# Function run by the Gradio chatbot.  This will perform sentiment analysis
# and call the correct LLM based on if the sentiment is negative 
def run(msg, history):

    sentiment = sentiment_analysis.score(msg)

    returned_message = ""

    #Initialize history if it's None
    if history is None:
        history = []

    #Append user message to the history
    #Append user message to the history imnediately
    history.append((returned_message, ""))
    yield history, None, history # Update the chat with user input


    # obtain the llm response
    # if the sentiment is negative, use the straight LLM, otherwise 
    # will utilize the openlibrary.org API
    
    # if we ever tripped the negativity flag, stay with that llm
    if neg_flag == True:
        sentiment == 'NEG'

    if sentiment == 'POS':
        returned_message =  run_api(msg, history)
    if sentiment == 'NEU':
        returned_message =  run_api(msg, history)
    if sentiment == 'NEG':
        print("Swithing to negative response.")
        returned_message = run_moody(msg, history)
    
    bot_message = returned_message

    #updaste the last message in history with the bot response
    history[-1] = (msg, bot_message)    

    # builds the text to speech component
    audio_file = text_to_speech(returned_message)

    yield history, audio_file, history

    return returned_message

In [10]:
# Create the Gradio interface
iface = gr.Interface(
    fn=run,
    inputs=[
        # gr.Textbox(label="Enter text", placeholder="Ask about books..."),
         gr.Textbox(label="Enter your message", placeholder="Type your question here..."),
        gr.State(value=[])  # To keep track of the chat history
    ],
    outputs=[
        gr.Chatbot(label="Chat History"),
        gr.Audio(label="Audio Response"),
        gr.State(value=[])
    ],
    title="Book Search Chatbot",
    description="A chatbot that searches books using Open Library API."
)

iface.launch()

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
